In [37]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
import tokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [38]:
import nltk
from nltk.tokenize.treebank import TreebankWordTokenizer
nltk.download('stopwords')
tokenizer = TreebankWordTokenizer()
stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [39]:
df = pd.read_csv('Twitter_Data.csv')

In [42]:
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [40]:
#CLEANING TEXTTT
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;].')
REPLACE_BY_NULL = re.compile('[#]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    #text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower()
    text = REPLACE_BY_NULL.sub('', text)# lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text 

In [43]:
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda,GRU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [45]:
df.dtypes

clean_text     object
category      float64
dtype: object

In [46]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['clean_text'].astype(str).values)

In [48]:
df['clean_text'] = tokenizer.texts_to_sequences(df['clean_text'].astype(str).values)
vocab_size = len(tokenizer.word_index) + 1

In [51]:
maxlen = 160
textdata = pad_sequences(df['clean_text'], maxlen=maxlen,padding='post')

In [52]:
textdata[0]

array([  42,    1,  307,   66, 1726, 1119,   40, 2378,    2, 1211,  205,
          2,  215,   32,  155,  100,   49,   69, 1068,  215,   50,    3,
          6,  546,    3,   50, 4179, 5582,    3, 2806,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [53]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [124]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(textdata,df.drop(columns=['clean_text']), test_size = 0.25, random_state=8)

In [125]:
y_train = pd.get_dummies(y_train['category'],dummy_na=False)
y_test = pd.get_dummies(y_test['category'],dummy_na=False)

In [126]:
y_train.shape

(122235, 3)

In [132]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, SpatialDropout1D, LSTM, GRU,Conv1D
#from tensorflow.keras.layers import add,concatenate,Conv1DMaxPooling1D,
from tensorflow.keras.layers import Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D,concatenate

In [138]:
input_layer1 = Input(shape=(textdata.shape[1],) ,name='x1')
input_layer2 = Input(shape=(1,), name='x2')
embed_layer = Embedding(vocab_size, embedding_matrix.shape[1],   weights=[embedding_matrix], input_length=maxlen , trainable=False)(input_layer1)
x = Bidirectional(GRU(64, return_sequences=True, dropout=0.5,recurrent_dropout=0.5))(embed_layer)
x = Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)

x = concatenate([ avg_pool, max_pool])

preds = Dense((3), activation="softmax")(x)

model = Model(input_layer1, preds)
model.compile(loss = 'categorical_crossentropy', optimizer='adam', 
              #metrics=['binary_crossentropy']
              metrics=['acc'])

print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 x1 (InputLayer)                [(None, 160)]        0           []                               
                                                                                                  
 embedding_23 (Embedding)       (None, 160, 100)     11368000    ['x1[0][0]']                     
                                                                                                  
 bidirectional_2 (Bidirectional  (None, 160, 128)    63744       ['embedding_23[0][0]']           
 )                                                                                                
                                                                                                  
 conv1d_2 (Conv1D)              (None, 158, 64)      24640       ['bidirectional_2[0][0]']  

In [139]:
history = model.fit(X_train,y_train, batch_size=32, epochs=5, verbose=1, validation_split=0.2)   

Epoch 1/5
3056/3056 [==============================] - 561s 182ms/step - loss: 0.7381 - acc: 0.6768 - val_loss: 0.5787 - val_acc: 0.7689
Epoch 2/5
3056/3056 [==============================] - 557s 182ms/step - loss: 0.6014 - acc: 0.7544 - val_loss: 0.4645 - val_acc: 0.8284
Epoch 3/5
3056/3056 [==============================] - 560s 183ms/step - loss: 0.5244 - acc: 0.7951 - val_loss: 0.3990 - val_acc: 0.8612
Epoch 4/5
3056/3056 [==============================] - 559s 183ms/step - loss: 0.4756 - acc: 0.8218 - val_loss: 0.3698 - val_acc: 0.8789
Epoch 5/5
3056/3056 [==============================] - 561s 183ms/step - loss: 0.4463 - acc: 0.8366 - val_loss: 0.3520 - val_acc: 0.8873


In [140]:
model.save('NLP_model.h5')

In [141]:
pred = model.predict(X_test)
pred = np.round(pred)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.8803534175972512